# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import os

print(os.uname())

# In the cloud environment?
if 'root' in os.environ['HOME']:
    UENVPATH = '/data/'
    !pip -q install /home/workspace/python

# In the standalone environment?
if 'ferenc' in os.environ['HOME']:
    UENVPATH = '/home/ferenc/Python/rl/udadrl/data/'


from unityagents import UnityEnvironment
import numpy as np

import pprint as pp

posix.uname_result(sysname='Linux', nodename='Anubis-Linux', release='5.4.0-42-generic', version='#46-Ubuntu SMP Fri Jul 10 00:24:02 UTC 2020', machine='x86_64')


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
# ONE Agent, Standalone
#UENVCHOICE = 'Reacher_Linux/Reacher.x86_64'

# TWENTY Agents, Standalone
#UENVCHOICE = 'Reacher_Linux_20/Reacher.x86_64'

# ONE Agent, Cloud, No-Visuals 
#UENVCHOICE = 'Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64'

# TWENTY Agents, Cloud, No-Visuals 
UENVCHOICE = 'Reacher_Linux_NoVis/Reacher.x86_64'

env = UnityEnvironment( file_name=os.path.join( UENVPATH, UENVCHOICE ) )




INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
# Some reminders about lambda functions
# Thx2: https://realpython.com/python-lambda/
# Thx2: https://thispointer.com/python-how-to-use-if-else-elif-in-lambda-functions/


env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)

epc = 0
print(f'We have {num_agents} active agent' + ( lambda x: 's' if x>1 else '' )(num_agents) )

while True:
    epc += 1
    #print(f'\r#{epc}', end = ' ')
    print(f'\r#{str(epc).rjust(6)}', end = ' ')
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    rewards = env_info.rewards
    nprewards = np.array(rewards)
    scores += rewards                         # update the score (for each agent)
    if np.sum(nprewards) > 0:
        #print(f'\r#{epc} Rewards = {rewards}')
        maskagent = nprewards > 0
        agentbefore = False
        for (nagent, action) in enumerate(actions):
            if maskagent[nagent]:
                if agentbefore:
                    print('\r#' + '&'.rjust(6), end = ' ')
                print(' -> Agent {:0>2d} got reward {:+.5f} for action: {}'.format(nagent+1, rewards[nagent], action))
                agentbefore = True
                #pp.pprint(list(action))
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('#')
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

We have 20 active agents
#    73  -> Agent 13 got reward +0.01000 for action: [ 0.3036464   1.          1.         -0.84985236]
#    74  -> Agent 13 got reward +0.04000 for action: [0.10844572 0.72525319 0.22602293 0.24358772]
#    75  -> Agent 13 got reward +0.04000 for action: [-0.52222201  0.84929896  1.         -0.78967267]
#    76  -> Agent 13 got reward +0.04000 for action: [-0.34437304  1.          0.98755469  0.41498856]
#    77  -> Agent 13 got reward +0.04000 for action: [-0.01114446  0.07020575  1.         -0.42085115]
#    78  -> Agent 13 got reward +0.04000 for action: [-0.64089699 -0.9768156   0.79347484  0.43360615]
#    79  -> Agent 13 got reward +0.04000 for action: [ 1.          0.82747337  0.64510842 -0.07910951]
#    80  -> Agent 13 got reward +0.01000 for action: [-1.          0.28461436 -0.57149256 -0.34046246]
#   145  -> Agent 20 got reward +0.01000 for action: [ 0.3610598  -0.81480258  0.29801802 -1.        ]
#   146  -> Agent 20 got reward +0.04000 for action:

### When finished, you can close the environment.

In [6]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

#### Tracing the error in the 'mya2cnet' module code refactoring below

In [7]:
# TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! 
#import pdb; pdb.set_trace()

import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(20200808) # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!

# Format: IN_Num [Layer 1] (OUT_Num = IN_Num) [Layer 2] OUT_Num = ...
HIDDEN_DIMS_DEFAULT = {
    'shared' : (512, 512, 256, 256),
    'actor' : (256, 128, 128, 64),
    'critic' : (256, 128, 128, 64)
}
hidden_dims = HIDDEN_DIMS_DEFAULT

hlayers = dict()

hlayers['shared'] = nn.ModuleList()
hlayers['actor'] = nn.ModuleList()
hlayers['critic'] = nn.ModuleList()




In [8]:
# Input layer
input_layer = nn.Linear( 33, hidden_dims['shared'][0] )

In [9]:
input_layer

Linear(in_features=33, out_features=512, bias=True)

In [10]:
# Hidden layers shared
for i in range( len(hidden_dims['shared']) -1 ):
    hlayers['shared'].append( nn.Linear( hidden_dims['shared'][i], hidden_dims['shared'][i+1] ) )

In [11]:
hlayers['shared']

ModuleList(
  (0): Linear(in_features=512, out_features=512, bias=True)
  (1): Linear(in_features=512, out_features=256, bias=True)
  (2): Linear(in_features=256, out_features=256, bias=True)
)

In [12]:
# Actor layers
for i in range( len(hidden_dims['actor']) ):
    #import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
    if i == 0:
        hlayers['actor'].append( nn.Linear( hidden_dims['shared'][-1], hidden_dims['actor'][i] ) )
    else:
        # hlayers['actor'].append( nn.Linear( hidden_dims['actor'][i], hidden_dims['actor'][i+1] ERROR !!!
        hlayers['actor'].append( nn.Linear( hidden_dims['actor'][i-1], hidden_dims['actor'][i] ) )
    #print( i, hlayers['actor'] ) # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
        
actor_out_layer = nn.Linear( hidden_dims['actor'][-1], 4 )

In [13]:
hlayers['actor']

ModuleList(
  (0): Linear(in_features=256, out_features=256, bias=True)
  (1): Linear(in_features=256, out_features=128, bias=True)
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): Linear(in_features=128, out_features=64, bias=True)
)

In [14]:
actor_out_layer

Linear(in_features=64, out_features=4, bias=True)

In [15]:
#Critic layers
for i in range( len(hidden_dims['critic']) ):
    if i == 0:
        hlayers['critic'].append( nn.Linear( hidden_dims['shared'][-1], hidden_dims['critic'][i] ) )
    else:
        hlayers['critic'].append( nn.Linear( hidden_dims['critic'][i-1], hidden_dims['critic'][i] ) )
critic_out_layer = nn.Linear( hidden_dims['critic'][-1], 1 )

In [16]:
hlayers['critic']

ModuleList(
  (0): Linear(in_features=256, out_features=256, bias=True)
  (1): Linear(in_features=256, out_features=128, bias=True)
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): Linear(in_features=128, out_features=64, bias=True)
)

In [17]:
critic_out_layer

Linear(in_features=64, out_features=1, bias=True)

In [18]:
# Prevents non Pytorch Tensor Object entering the processing stream
def torch_format(state):
    x = state
    if not isinstance(x, torch.Tensor):
        x = torch.tensor(x, dtype=torch.float32)
        if len(x.size()) == 1:
            x = x.unsqueeze(0)
    return x

In [19]:
def forward(state):
    check_tensor = lambda x: isinstance(x, torch.Tensor)
    #last_x_shared_actor = True
    #last_x_shared_critic = True
    x_act = True 
    x_crit = True

    
    # BEGIN! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
    #x_act = 1 
    #x_crit = 1 
    ddims = lambda x: x.size() if check_tensor(x) else 'NoTorch'
    # END! Debug! Debug! Debug! Debug! Debug! Debug! Debug!

    x = torch_format(state)
    x = F.relu(  input_layer(x) )
    for label in ['shared', 'actor', 'critic']:
        #for hlayer in  hlayers[label]:
        for (dc, hlayer) in  enumerate(hlayers[label]): # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
            #print(f'Label: {label}; HLayer: {dc} #####') # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
            if label == 'shared':
                x = F.relu(  hlayer(x) )
            if label == 'actor':
                #import pdb; pdb.set_trace()
                #if check_tensor(last_x_shared_actor):
                #    x_act = F.relu(  hlayer(last_x_shared_actor) ).clone()
                #    last_x_shared_actor = False
                #else:
                #print(hlayer) # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
                x_act = F.relu(  hlayer(x_act) )
            if label == 'critic':
                #if check_tensor(last_x_shared_critic):
                #    x_crit = F.relu(  hlayer(last_x_shared_critic) ).clone()
                #    last_x_shared_critic = False
                #else:
                x_crit = F.relu(  hlayer(x_crit) )
            # BEGIN! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
            #print(f'Types: x is {type(x)}; x_act is {type(x_act)}; last_x_shared_actor is {type(last_x_shared_actor)}')
            #print(f'Dims: x is {ddims(x)};' +\
            #      f' x_act is {ddims(x_act)};' +\
            #      f' last_x_shared_actor is {ddims(last_x_shared_actor)}')
            # END! Debug! Debug! Debug! Debug! Debug! Debug! Debug!

        # Thx2: https://discuss.pytorch.org/t/copy-deepcopy-vs-clone/55022
        if ( type(x_act) == bool ):
            x_act = x.clone()  # Create an Inplace copy...
        if ( type(x_crit) == bool ):
            x_crit = x.clone() # ...after processing shared layers

    return  actor_out_layer(x_act),  critic_out_layer(x_crit) 

#### Test if states are propagated through the debug network

In [20]:
for (dc, st) in enumerate(states):
    a,b = forward(st)
    print(f'({dc+1}) -> Actor: {a}')
    print(f'({dc+1}) -> Critic: {b}')

(1) -> Actor: tensor(1.00000e-02 *
       [[ 4.6543,  4.9756, -4.4953,  4.2128]])
(1) -> Critic: tensor(1.00000e-02 *
       [[ 4.7572]])
(2) -> Actor: tensor(1.00000e-02 *
       [[ 4.7037,  4.9775, -4.4215,  4.1041]])
(2) -> Critic: tensor(1.00000e-02 *
       [[ 4.8767]])
(3) -> Actor: tensor(1.00000e-02 *
       [[ 4.7006,  4.9798, -4.4235,  4.1018]])
(3) -> Critic: tensor(1.00000e-02 *
       [[ 4.8752]])
(4) -> Actor: tensor(1.00000e-02 *
       [[ 4.6981,  4.9901, -4.3603,  4.1175]])
(4) -> Critic: tensor(1.00000e-02 *
       [[ 4.8696]])
(5) -> Actor: tensor(1.00000e-02 *
       [[ 4.7117,  4.9911, -4.4272,  4.1191]])
(5) -> Critic: tensor(1.00000e-02 *
       [[ 4.8610]])
(6) -> Actor: tensor(1.00000e-02 *
       [[ 4.6864,  4.9768, -4.4898,  4.2664]])
(6) -> Critic: tensor(1.00000e-02 *
       [[ 4.7746]])
(7) -> Actor: tensor(1.00000e-02 *
       [[ 4.7017,  5.0059, -4.4374,  4.2481]])
(7) -> Critic: tensor(1.00000e-02 *
       [[ 4.8414]])
(8) -> Actor: tensor(1.00000e-02 *

#### Test if states are propagated through the imported network

In [21]:
#from mya2cnet import A2CNetwork
import mya2cnet

# Thx2: https://emacs.stackexchange.com/a/13483
#import imp
#imp.reload(mya2cnet)

tstnet = mya2cnet.A2CNetwork(33, 4)

In [22]:
for (dc, st) in enumerate(states):
    #import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
    a,b = tstnet.forward(st)
    print(f'({dc+1}) -> Actor: {a}')
    print(f'({dc+1}) -> Critic: {b}')

(1) -> Actor: tensor(1.00000e-02 *
       [[-1.2575,  3.2509,  3.0744,  0.9772]])
(1) -> Critic: tensor(1.00000e-02 *
       [[-3.3071]])
(2) -> Actor: tensor(1.00000e-02 *
       [[-1.3777,  3.3048,  2.9574,  1.0498]])
(2) -> Critic: tensor(1.00000e-02 *
       [[-3.3378]])
(3) -> Actor: tensor(1.00000e-02 *
       [[-1.3587,  3.2907,  2.9713,  1.0452]])
(3) -> Critic: tensor(1.00000e-02 *
       [[-3.3433]])
(4) -> Actor: tensor(1.00000e-02 *
       [[-1.2888,  3.2493,  3.0360,  1.0206]])
(4) -> Critic: tensor(1.00000e-02 *
       [[-3.3420]])
(5) -> Actor: tensor(1.00000e-02 *
       [[-1.3718,  3.3208,  2.9547,  1.0424]])
(5) -> Critic: tensor(1.00000e-02 *
       [[-3.2910]])
(6) -> Actor: tensor(1.00000e-02 *
       [[-1.2530,  3.2534,  3.0799,  0.9857]])
(6) -> Critic: tensor(1.00000e-02 *
       [[-3.2941]])
(7) -> Actor: tensor(1.00000e-02 *
       [[-1.2849,  3.2506,  3.0638,  1.0235]])
(7) -> Critic: tensor(1.00000e-02 *
       [[-3.3235]])
(8) -> Actor: tensor(1.00000e-02 *